<a href="https://colab.research.google.com/github/pasindu-pr/qdrant-q-and-a-on-pdf/blob/main/LLM%20Sherpa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install openai qdrant-client llmsherpa

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.1/267.1 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.2/223.2 kB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 8.8 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-me

In [2]:
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams
from qdrant_client.http import models
import openai
import os
from openai import AzureOpenAI

from google.colab import userdata


openai.api_key = "sk-xxxxxxxxxxxxxxxxxxxxxxxxxx"

client = AzureOpenAI(
  api_key = userdata.get("AZURE_OPENAI_API_KEY"),
  api_version = "2024-02-01",
  azure_endpoint = userdata.get("AZURE_OPENAI_ENDPOINT")
)


In [3]:
qdrant_client = QdrantClient(
    host=userdata.get('QDRANT_HOST'),
    api_key=userdata.get('QDRANT_KEY'),
)

In [4]:
qdrant_client.recreate_collection(
    collection_name="mycollection",
    vectors_config=models.VectorParams(size=3072, distance=models.Distance.COSINE),
)
print("Create collection reponse:", qdrant_client)

Create collection reponse: <qdrant_client.qdrant_client.QdrantClient object at 0x7ce01b383220>


In [5]:
collection_info = qdrant_client.get_collection(collection_name="mycollection")

print("Collection info:", collection_info)

Collection info: status=<CollectionStatus.GREEN: 'green'> optimizer_status=<OptimizersStatusOneOf.OK: 'ok'> vectors_count=0 indexed_vectors_count=0 points_count=0 segments_count=2 config=CollectionConfig(params=CollectionParams(vectors=VectorParams(size=3072, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=None), shard_number=1, sharding_method=None, replication_factor=1, write_consistency_factor=1, read_fan_out_factor=None, on_disk_payload=True, sparse_vectors=None), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=False, payload_m=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=None, indexing_threshold=20000, flush_interval_sec=5, max_optimization_threads=None), wal_config=WalConfig(wal_capacity_mb=32, wal_segments_ahead=0), quantization_config=None) payload_schema={}


In [6]:
from llmsherpa.readers import LayoutPDFReader

llmsherpa_api_url = "https://readers.llmsherpa.com/api/document/developer/parseDocument?renderFormat=all"
pdf_url = "/content/InjuredWorkerGuidebookCalifornia.pdf" # also allowed is a file path e.g. /home/downloads/xyz.pdf
pdf_reader = LayoutPDFReader(llmsherpa_api_url)
doc = pdf_reader.read_pdf(pdf_url)

In [10]:
from qdrant_client.http.models import PointStruct

points=[]
i = 1
for chunk in doc.chunks():
    i += 1
    response = client.embeddings.create(
        input=chunk.to_context_text(),
        model="embedding-model"
    )
    embeddings = response.data[0].embedding

    points.append(PointStruct(id=i, vector=embeddings, payload={"text": chunk.to_context_text()}))

operation_info = qdrant_client.upsert(
    collection_name="mycollection",
    points=points
)

print("Operation info:", operation_info)

UnexpectedResponse: Unexpected Response: 400 (Bad Request)
Raw response content:
b'{"status":{"error":"Payload error: JSON payload (34447460 bytes) is larger than allowed (limit: 33554432 bytes)."},"time":0.0}'

In [ ]:
operation_info = qdrant_client.upsert(
    collection_name="mycollection",
    points=points
)

print("Operation info:", operation_info)

In [ ]:
def create_answer_with_context(query):
    response = client.embeddings.create(
        input="What is the GROSS WAGES BEFORE INJURY for different occupations?",
        model="embedding-model"
    )
    embeddings = response.data[0].embedding

    search_result = qdrant_client.search(
        collection_name="mycollection",
        query_vector=embeddings,
        limit=5
    )

    prompt = "Context:\n"
    for result in search_result:
        prompt += result.payload['text'] + "\n---\n"
    prompt += "Question:" + query + "\n---\n" + "Answer:"

    print("----PROMPT START----")
    print(":", prompt)
    print("----PROMPT END----")

    completion = client.chat.completions.create(
        model=userdata.get("CHAT_MODEL"),
        messages=[
            {"role": "user", "content": prompt}
        ]
        )

    return completion.choices[0].message.content



In [ ]:

input = "What is the GROSS WAGES BEFORE INJURY for different occupations"
answer = create_answer_with_context(input)
print(answer)
